News Stance Detection

import package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [2]:
#self defined function from .py files
import clean as cl
import tfidf as ti
import feature as fe
import language_model as lm
import model as md

load data

In [3]:
train_bodies = pd.read_csv('./fnc-1/train_bodies.csv')
train_stances = pd.read_csv('./fnc-1/train_stances.csv')
test_bodies = pd.read_csv('./fnc-1/competition_test_bodies.csv')
test_stances = pd.read_csv('./fnc-1/competition_test_stances.csv')

FileNotFoundError: File b'./fnc-1/train_bodies.csv' does not exist

In [ ]:
train_bodies.head()

In [ ]:
train_bodies.shape

In [ ]:
train_stances.head()

In [ ]:
train_stances.shape

In [ ]:
test_bodies.shape

In [ ]:
test_stances.shape

clean data

In [ ]:
clean_stances = train_stances.copy()
clean_bodies = train_bodies.copy()

In [ ]:
clean_headline = [cl.clean_sentence(sentence) for sentence in clean_stances['Headline']]   
clean_article = [cl.clean_sentence(sentence) for sentence in clean_bodies['articleBody']]  

In [ ]:
clean_stances['Headline'] = clean_headline
clean_bodies['articleBody'] = clean_article

In [ ]:
#test set
clean_test_stances = test_stances.copy()
clean_test_bodies = test_bodies.copy()

In [ ]:
clean_test_headline = [cl.clean_sentence(sentence) for sentence in clean_test_stances['Headline']]   
clean_test_article = [cl.clean_sentence(sentence) for sentence in clean_test_bodies['articleBody']]  

In [ ]:
clean_test_stances['Headline'] = clean_test_headline
clean_test_bodies['articleBody'] = clean_test_article

In [ ]:
clean_stances.head()

In [ ]:
clean_stances.shape

In [ ]:
clean_bodies.head()

In [ ]:
clean_bodies.shape

In [ ]:
clean_test_stances.head()

In [ ]:
clean_test_bodies.head()

observe unique headlines and bodies

In [ ]:
uni_head = train_stances['Headline'].unique()

In [ ]:
uni_body = train_bodies['articleBody'].unique()

In [ ]:
clean_uni_head = [cl.clean_sentence(sentence) for sentence in uni_head]   
clean_uni_body = [cl.clean_sentence(sentence) for sentence in uni_body]  

In [ ]:
len(clean_uni_body)

In [ ]:
len(clean_uni_head)

In [ ]:
clean_uni_all = clean_uni_head + clean_uni_body

In [ ]:
len(clean_uni_all)

merge body and stance

In [ ]:
clean_merge = pd.merge(clean_stances, clean_bodies, how='left', on=['Body ID', 'Body ID'])

In [ ]:
clean_merge.head()

In [ ]:
sum(clean_merge.Headline == clean_stances.Headline)

In [ ]:
clean_merge.shape

In [ ]:
clean_test_merge = pd.merge(clean_test_stances, clean_test_bodies, how='left', on=['Body ID', 'Body ID'])

In [ ]:
clean_test_merge.head()

In [ ]:
sum(clean_test_merge.Headline == clean_test_stances.Headline)

In [ ]:
clean_test_merge.shape

###  subtask 2. vector representation and cosine  similarity

build vocabulary

In [ ]:
#all voc
voc_all = []
for sentence in clean_uni_all:
    voc_all = voc_all + sentence

In [ ]:
voc_all = set(voc_all)

In [ ]:
len(voc_all)

In [ ]:
#add index
inx_voc_all = dict(zip(voc_all,range(len(voc_all))))

tf-idf

In [ ]:
'''
#idf was calculated and saved
idf = ti.calculate_idf(voc_all,clean_uni_all)

import json
# Writing idf
with open('idf.json', 'w') as f:
    json.dump(idf, f)
'''

In [ ]:
import json
# Reading idf
with open('idf.json', 'r') as f:
    idf = json.load(f)

feature selection for tfidf

In [ ]:
voc_key = []
for sentence in clean_uni_head:
    voc_key = voc_key + ti.tfidf_filter(3, sentence, idf)
for sentence in clean_uni_body:
    voc_key = voc_key + ti.tfidf_filter(30, sentence, idf)

In [ ]:
voc_key = set(voc_key)

In [ ]:
len(voc_key)

In [ ]:
#add index
inx_voc_key = dict(zip(voc_key,range(len(voc_key))))

word2vec

In [ ]:
clean_all = clean_merge['Headline']+clean_merge['articleBody']

In [ ]:
len(clean_all)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
dim = 200
w2v_model = Word2Vec(sentences=clean_all, min_count=1, size = dim)

In [ ]:
voc_w2v = list(w2v_model.wv.vocab.keys()) 
len(voc_w2v)

In [ ]:
def word2vec_vector(sentence, model, voc, dim):
    vector = np.zeros(dim)
    for word in sentence:
        if word in voc:
            vector = np.add(vector, model.wv.word_vec(word))
    vector = vector / np.linalg.norm(vector)
    return vector  

calculate cosine

In [ ]:
all_merge = clean_merge.copy()

In [ ]:
all_merge[0:5]

In [ ]:
len(all_merge)

In [ ]:
all_merge['cos_sim'] = 0.00

In [ ]:
col = 4
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector(h1, idf, inx_voc_all)
    vb1 = ti.calculate_tfidf_vector(b1, idf, inx_voc_all)
    all_merge.iat[i,col] = fe.cos_similar(vh1,vb1)
#     print(cos_similar(vh1,vb1))

In [ ]:
all_merge['cos_sim_key'] = 0.0

In [ ]:
col = 5
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_merge.iat[i,col] = fe.cos_similar(vh1,vb1)

In [ ]:
all_merge['cos_sim_w2v'] = 0.0

In [ ]:
col = 6
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_merge.iat[i,col] = fe.cos_similar(vh1,vb1)

test set

In [ ]:
all_test_merge = clean_test_merge.copy()

In [ ]:
all_test_merge[0:2]

In [ ]:
all_test_merge['cos_sim'] = 0.00

In [ ]:
col = 4
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_all, voc_all)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_all, voc_all)
    all_test_merge.iat[i,col] = fe.cos_similar(vh1,vb1)

In [ ]:
all_test_merge['cos_sim_key'] = 0.0

In [ ]:
col = 5
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_test_merge.iat[i,col] = fe.cos_similar(vh1,vb1)

In [ ]:
all_test_merge['cos_sim_w2v'] = 0.0

In [ ]:
col = 6
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_test_merge.iat[i,col] = fe.cos_similar(vh1,vb1)

###  subtask 3. language model

In [ ]:
def create_unigram(sentence):
    unigram = lm.Ngram_lm(sentence,1)
    return unigram

In [ ]:
def create_bigram(sentence):
    alpha1=0.7
    unigram = lm.Ngram_lm(sentence,1)
    bigram = lm.Ngram_lm(sentence,2)
    bi_unigram=lm.Inter_lm(bigram,unigram,alpha1)
    return bi_unigram

In [ ]:
all_merge['kl_head_uni'] = 0.0

In [ ]:
#use models to predict head
col = 7
for i in range(len(all_merge)):
    h = all_merge.iat[i,0]
    b = all_merge.iat[i,3]
    lm1 = create_unigram(h)
    lm2 = create_unigram(b)
    p = lm.lm_vector(h, lm1)
    q = lm.lm_vector(h, lm2)
    all_merge.iat[i,col] = lm.kl_div(p, q)

In [ ]:
all_merge['kl_head_bi'] = 0.0

In [ ]:
#use models to predict head
col = 8
for i in range(len(all_merge)):
    h = all_merge.iat[i,0]
    b = all_merge.iat[i,3]
    lm1 = create_bigram(h)
    lm2 = create_bigram(b)
    p = lm.lm_vector(h, lm1)
    q = lm.lm_vector(h, lm2)
    all_merge.iat[i,col] = lm.kl_div(p, q)

test set

In [ ]:
all_test_merge['kl_head_uni'] = 0.0

In [ ]:
#use models to predict head
col = 7
for i in range(len(all_test_merge)):
    h = all_test_merge.iat[i,0]
    b = all_test_merge.iat[i,3]
    lm1 = create_unigram(h)
    lm2 = create_unigram(b)
    p = lm.lm_vector(h, lm1)
    q = lm.lm_vector(h, lm2)
    all_test_merge.iat[i,col] = lm.kl_div(p, q)

In [ ]:
all_test_merge['kl_head_bi'] = 0.0

In [ ]:
#use models to predict head
col = 8
for i in range(len(all_test_merge)):
    h = all_test_merge.iat[i,0]
    b = all_test_merge.iat[i,3]
    lm1 = create_bigram(h)
    lm2 = create_bigram(b)
    p = lm.lm_vector(h, lm1)
    q = lm.lm_vector(h, lm2)
    all_test_merge.iat[i,col] = lm.kl_div(p, q)

###  subtask 4. Propose and implement alternative features/distances

Euclidean distance

In [ ]:
all_merge['euc_dist'] = 0.0

In [ ]:
col = 9
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector(h1, idf, inx_voc_all)
    vb1 = ti.calculate_tfidf_vector(b1, idf, inx_voc_all)
    all_merge.iat[i,col] = fe.euc_dist(vh1,vb1)
#     print(cos_similar(vh1,vb1))

In [ ]:
all_merge['euc_dist_key'] = 0.0

In [ ]:
col = 10
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_merge.iat[i,col] = fe.euc_dist(vh1,vb1)

In [ ]:
all_merge['euc_dist_w2v'] = 0.0

In [ ]:
col = 11
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_merge.iat[i,col] = fe.euc_dist(vh1,vb1)

Spearman correlation 

In [ ]:
all_merge['spear_corr'] = 0.0

In [ ]:
col = 12
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector(h1, idf, inx_voc_all)
    vb1 = ti.calculate_tfidf_vector(b1, idf, inx_voc_all)
    all_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)
#     print(cos_similar(vh1,vb1))

In [ ]:
all_merge['spear_corr_key'] = 0.0

In [ ]:
col = 13
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)

In [ ]:
all_merge['spear_corr_w2v'] = 0.0

In [ ]:
col = 14
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)

same words

In [ ]:
all_merge['same_word'] = 0.0

In [ ]:
col = 15
for i in range(len(all_merge)):
    h1 = all_merge.iat[i,0]
    b1 = all_merge.iat[i,3]
    all_merge.iat[i,col] = fe.same_word(h1,b1)

test set

Euclidean distance

In [ ]:
all_test_merge['euc_dist'] = 0.0

In [ ]:
col = 9
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_all, voc_all)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_all, voc_all)
    all_test_merge.iat[i,col] = fe.euc_dist(vh1,vb1)
#     print(cos_similar(vh1,vb1))

In [ ]:
all_test_merge['euc_dist_key'] = 0.0

In [ ]:
col = 10
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_test_merge.iat[i,col] = fe.euc_dist(vh1,vb1)

In [ ]:
all_test_merge['euc_dist_w2v'] = 0.0

In [ ]:
col = 11
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_test_merge.iat[i,col] = fe.euc_dist(vh1,vb1)

Spearman correlation 

In [ ]:
all_test_merge['spear_corr'] = 0.0

In [ ]:
col = 12
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_all, voc_all)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_all, voc_all)
    all_test_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)
#     print(cos_similar(vh1,vb1))

In [ ]:
all_test_merge['spear_corr_key'] = 0.0

In [ ]:
col = 13
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = ti.calculate_tfidf_vector_key(h1, idf, inx_voc_key, voc_key)
    vb1 = ti.calculate_tfidf_vector_key(b1, idf, inx_voc_key, voc_key)
    all_test_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)

In [ ]:
all_test_merge['spear_corr_w2v'] = 0.0

In [ ]:
col = 14
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    vh1 = word2vec_vector(h1, w2v_model,voc_w2v,dim)
    vb1 = word2vec_vector(b1, w2v_model,voc_w2v,dim)
    all_test_merge.iat[i,col] = fe.spearman_corr(vh1,vb1)

same words

In [ ]:
all_test_merge['same_word'] = 0.0

In [ ]:
col = 15
for i in range(len(all_test_merge)):
    h1 = all_test_merge.iat[i,0]
    b1 = all_test_merge.iat[i,3]
    all_test_merge.iat[i,col] = fe.same_word(h1,b1)

###  subtask 5.analysis the distributions

In [ ]:
# all_merge = pd.read_csv('all_merge.csv')

In [ ]:
all_merge[0:2]

In [ ]:
all_merge[['Stance','euc_dist','euc_dist_key','euc_dist_w2v','spear_corr','spear_corr_key','spear_corr_w2v','same_word']][0:2]


In [ ]:
all_merge.shape

In [ ]:
# all_merge.to_csv('all_merge.csv')

In [ ]:
all_merge_unrelated = all_merge[all_merge['Stance'] == 'unrelated']
all_merge_agree = all_merge[all_merge['Stance'] == 'agree']
all_merge_disagree = all_merge[all_merge['Stance'] == 'disagree']
all_merge_discuss = all_merge[all_merge['Stance'] == 'discuss']

In [ ]:
all_merge_unrelated.shape[0]+all_merge_agree.shape[0]+all_merge_disagree.shape[0]+all_merge_discuss.shape[0]

In [ ]:
all_merge.describe()

In [ ]:
all_merge_unrelated.describe()

In [ ]:
all_merge_agree.describe()

In [ ]:
all_merge_disagree.describe()

In [ ]:
all_merge_discuss.describe()

In [ ]:
feature_name = 'cos_sim'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'cos_sim_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1000)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'cos_sim_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'kl_head_uni'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')
plt.ylim(0,1000)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'kl_head_bi'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'euc_dist'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'euc_dist_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'euc_dist_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'spear_corr'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'spear_corr_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="olive", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'spear_corr_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'same_word'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_merge_agree[feature_name], kde=False,  bins=100, color="olive", label='Agree')
sns.distplot(all_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

test set

In [ ]:
all_test_merge[0:2]

In [ ]:
all_test_merge_copy = all_test_merge.copy()

In [ ]:
all_test_merge.shape

In [ ]:
(all_test_merge.dropna()).shape

In [ ]:
#There are some out of vocabulary words are reset to 0
all_test_merge = all_test_merge.fillna(value=0)

In [ ]:
all_test_merge.shape

In [ ]:
(all_test_merge.dropna()).shape

In [ ]:
# all_test_merge.to_csv('all_test_merge.csv')

In [ ]:
# all_test_merge = pd.read_csv('all_test_merge.csv')

In [ ]:
all_test_merge_unrelated = all_test_merge[all_test_merge['Stance'] == 'unrelated']
all_test_merge_agree = all_test_merge[all_test_merge['Stance'] == 'agree']
all_test_merge_disagree = all_test_merge[all_test_merge['Stance'] == 'disagree']
all_test_merge_discuss = all_test_merge[all_test_merge['Stance'] == 'discuss']

In [ ]:
all_test_merge.describe()

In [ ]:
all_test_merge_unrelated.describe()

In [ ]:
all_test_merge_agree.describe()

In [ ]:
all_test_merge_disagree.describe()

In [ ]:
all_test_merge_discuss.describe()

In [ ]:
feature_name = 'cos_sim'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(np.nan_to_num(all_test_merge_unrelated[feature_name]), kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
#bad feature
feature_name = 'cos_sim_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'cos_sim_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'kl_head_uni'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'kl_head_bi'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
#bad feature
feature_name = 'euc_dist'
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="olive", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
#bad feature
feature_name = 'euc_dist_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="olive", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'euc_dist_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'spear_corr'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
#bad feature
feature_name = 'spear_corr_key'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'spear_corr_w2v'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="pink", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
feature_name = 'same_word'
# Plot the frequency distribution (Pearson Correlation Coefficient)
plt.figure(figsize = (10, 10))

sns.distplot(all_test_merge_unrelated[feature_name], kde=False,  bins=100, color="blue",  label = 'Unrelated')
sns.distplot(all_test_merge_discuss[feature_name], kde=False,  bins=100, color="yellow",  label = 'Discuss')
sns.distplot(all_test_merge_agree[feature_name], kde=False,  bins=100, color="olive", label='Agree')
sns.distplot(all_test_merge_disagree[feature_name], kde=False,  bins=100, color="red", label = 'Disagree')

plt.ylim(0,1500)
# plt.title('The Distribution of the Pearson Correlation Coefficients for the Four Class')
plt.xlabel(feature_name)
plt.ylabel('Frequency')
plt.legend()
plt.show()

Insert one vs all label

In [ ]:
data = all_merge.copy()

In [ ]:
data[0:2]

In [ ]:
data['lin_unrelated'] = -1
data['lin_agree'] = -1
data['lin_disagree'] = -1
data['lin_discuss'] = -1

In [ ]:
data['lr_unrelated'] = 0
data['lr_agree'] = 0
data['lr_disagree'] = 0
data['lr_discuss'] = 0

In [ ]:
data['lin_unrelated'][data['Stance'] == 'unrelated'] = 1
data['lin_agree'][data['Stance'] == 'agree'] = 1
data['lin_disagree'][data['Stance'] == 'disagree'] = 1
data['lin_discuss'][data['Stance'] == 'discuss'] = 1

data['lr_unrelated'][data['Stance'] == 'unrelated'] = 1
data['lr_agree'][data['Stance'] == 'agree'] = 1
data['lr_disagree'][data['Stance'] == 'disagree'] = 1
data['lr_discuss'][data['Stance'] == 'discuss'] = 1

In [ ]:
# data.to_csv('data.csv')

In [ ]:
# data = pd.read_csv('data.csv')

In [ ]:
test_data = all_test_merge.copy()

In [ ]:
test_data[0:2]

In [ ]:
test_data['lin_unrelated'] = -1
test_data['lin_agree'] = -1
test_data['lin_disagree'] = -1
test_data['lin_discuss'] = -1

In [ ]:
test_data['lr_unrelated'] = 0
test_data['lr_agree'] = 0
test_data['lr_disagree'] = 0
test_data['lr_discuss'] = 0

In [ ]:
test_data['lin_unrelated'][test_data['Stance'] == 'unrelated'] = 1
test_data['lin_agree'][test_data['Stance'] == 'agree'] = 1
test_data['lin_disagree'][test_data['Stance'] == 'disagree'] = 1
test_data['lin_discuss'][test_data['Stance'] == 'discuss'] = 1

test_data['lr_unrelated'][test_data['Stance'] == 'unrelated'] = 1
test_data['lr_agree'][test_data['Stance'] == 'agree'] = 1
test_data['lr_disagree'][test_data['Stance'] == 'disagree'] = 1
test_data['lr_discuss'][test_data['Stance'] == 'discuss'] = 1

In [ ]:
# test_data.to_csv('test_data.csv')

In [ ]:
# test_data = pd.read_csv('test_data.csv')

###  subtask 1. training set and validation subset split

In [ ]:
#shuffle the data
data = data.sample(frac=1)

In [ ]:
data[0:2]

In [ ]:
data.shape

In [ ]:
#split the training set and validation set
cut = round(49972*0.9)
train = data[0:cut]
valid = data[cut:49972]

In [ ]:
train.shape

In [ ]:
valid.shape

ratios of the four classes

ratios of the four classes in the whole set

In [ ]:
train_stances[train_stances['Stance'] == 'agree'].shape[0]/train_stances.shape[0]

In [ ]:
train_stances[train_stances['Stance'] == 'disagree'].shape[0]/train_stances.shape[0]

In [ ]:
train_stances[train_stances['Stance'] == 'discuss'].shape[0]/train_stances.shape[0]

In [ ]:
train_stances[train_stances['Stance'] == 'unrelated'].shape[0]/train_stances.shape[0]

ratios of the four classes in the training set

In [ ]:
train[train['Stance'] == 'agree'].shape[0]/train.shape[0]

In [ ]:
train[train['Stance'] == 'disagree'].shape[0]/train.shape[0]

In [ ]:
train[train['Stance'] == 'discuss'].shape[0]/train.shape[0]

In [ ]:
train[train['Stance'] == 'unrelated'].shape[0]/train.shape[0]

ratios of the four classes in the validation set

In [ ]:
valid[valid['Stance'] == 'agree'].shape[0]/valid.shape[0]

In [ ]:
valid[valid['Stance'] == 'disagree'].shape[0]/valid.shape[0]

In [ ]:
valid[valid['Stance'] == 'discuss'].shape[0]/valid.shape[0]

In [ ]:
valid[valid['Stance'] == 'unrelated'].shape[0]/valid.shape[0]

###  subtask 6-10. model implementation, evaluation and improvement (Linear, LR, XGBoost)

In [ ]:
def downsampling(df, model_name, r = 0.1):
    df1 = df[df[model_name]>0]
    df0 = df[df[model_name] <= 0]
    dfs = df0.sample(frac=r)
    sample = pd.concat([df1, dfs],axis=0)
    return sample

In [ ]:
def onevsall(y_pred):
    result = []
    _,m = y_pred.shape
    y_pred_class = (y_pred == np.max(y_pred, axis = 0))
    for i in range(m):
        label = ''
        if y_pred_class[0,i]:
            label = 'unrelated'
        elif y_pred_class[1,i]:
            label = 'agree'
        elif y_pred_class[2,i]:
            label = 'disagree'
        elif y_pred_class[3,i]:
            label = 'discuss'
        result.append(label)
        
    assert len(result) == m
    return result

In [ ]:
def onevsall_new(y_pred,t=[0,0,0,0]):
    result = []
    _,m = y_pred.shape #(4,m)
    for i in range(m):
        is_unrelated = False
        is_agree = False
        is_disagree = False
        is_discuss = False
        if y_pred[0,i] > t[0]:
            is_unrelated = True
        if y_pred[1,i] > t[1]:
            is_agree = True
        if y_pred[2,i] > t[2]:
            is_disagree = True
        if y_pred[3,i] > t[3]:
            is_discuss = True
        
        if is_disagree:
            label = 'disagree'
        elif is_agree:
            label = 'agree'
        elif is_discuss:
            label = 'discuss'
        elif is_unrelated:
            label = 'unrelated'
        
        result.append(label)
        
    assert len(result) == m
    return result

In [ ]:
train[0:2]

prepare X

In [ ]:
list(train.columns)

In [ ]:
col = list(train.columns)[4:16]

In [ ]:
col

In [ ]:
# xcols = ['cos_sim_key','cos_sim_w2v','kl_head_uni','euc_dist_w2v','spear_corr_w2v']
xcols = col

In [ ]:
train_X = train[xcols].as_matrix()
valid_X = valid[xcols].as_matrix()
test_X = test_data[xcols].as_matrix()

train_X = md.preprocess_X(train_X)
valid_X = md.preprocess_X(valid_X)
test_X = md.preprocess_X(test_X)

In [ ]:
m,n = train_X.shape
learning_rate = 0.1
iters = 500
ini_theta = np.zeros([n,1])

linear models

In [ ]:
theta_lin = []
cost_lin = []
model_names_lin = ['lin_unrelated','lin_agree','lin_disagree','lin_discuss']
#restore prediction on validation
y_pred_lin = np.zeros([4,len(valid_X)]) 

In [ ]:
#build 4 models
#unrelated
i = 0
model_name = model_names_lin[i]
print(model_name)
X = train_X
y = train[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_linear(X, y, ini_theta, learning_rate, iters)
# print(theta)
theta_lin.append(theta)
cost_lin.append(cost)
y_pred_lin[i] = (valid_X@theta).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0.2
y_pred_class = y_pred_lin[i] > threshold
valid_y_class = valid_y > 0
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lin[i])

In [ ]:
#agree
i = 1
model_name = model_names_lin[i]
print(model_name)
#downsampling
# sample = downsampling(train, model_name, r = 1)
sample = downsampling(train, model_name, r = 0.3)
X = sample[xcols].as_matrix()
X = md.preprocess_X(X)
y = sample[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_linear(X, y, ini_theta, learning_rate, iters)
# print(theta)
theta_lin.append(theta)
cost_lin.append(cost)
y_pred_lin[i] = (valid_X@theta).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0
y_pred_class = y_pred_lin[i] > threshold
valid_y_class = valid_y > 0
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lin[i])

In [ ]:
#disagree
i = 2
model_name = model_names_lin[i]
print(model_name)
#downsampling
# sample = downsampling(train, model_name, r = 1)
sample = downsampling(train, model_name, r = 0.05)
X = sample[xcols].as_matrix()
X = md.preprocess_X(X)
y = sample[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_linear(X, y, ini_theta, learning_rate, iters)
# print(theta)
theta_lin.append(theta)
cost_lin.append(cost)
y_pred_lin[i] = (valid_X@theta).reshape(len(valid_X))

In [ ]:
threshold = -0.3
y_pred_class = y_pred_lin[i] > threshold
valid_y_class = valid_y > 0
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lin[i])

In [ ]:
#discuss
i = 3
model_name = model_names_lin[i]
print(model_name)
X = train_X
y = train[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_linear(X, y, ini_theta, learning_rate, iters)
# print(theta)
theta_lin.append(theta)
cost_lin.append(cost)
y_pred_lin[i] = (valid_X@theta).reshape(len(valid_X))

In [ ]:
threshold = -0.1
y_pred_class = y_pred_lin[i] > threshold
valid_y_class = valid_y > 0
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lin[i])

In [ ]:
plt.figure(figsize = (15, 5))
plt.scatter(range(len(theta_lin[0])-1),np.abs(theta_lin[0][1:]), color = 'blue', label = 'unrelated')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lin[1][1:]), color = 'pink', label = 'agree')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lin[2][1:]), color = 'yellow', label = 'disagree')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lin[3][1:]), color = 'orange', label = 'discuss')
plt.xlabel('Features')
plt.ylabel('Theta Values of Linear Models')
plt.legend()
plt.show()

In [ ]:
t = [0.2, 0, -0.3, -0.1]

In [ ]:
valid_pred_lin = onevsall(y_pred_lin)

In [ ]:
sum(valid_pred_lin == valid['Stance'])/ 4997

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = valid['Stance']
y_pred = valid_pred_lin
print(classification_report(y_true, y_pred))

In [ ]:
valid_pred_lin = onevsall_new(y_pred_lin,t)

In [ ]:
sum(valid_pred_lin == valid['Stance'])/ 4997

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = valid['Stance']
y_pred = valid_pred_lin
print(classification_report(y_true, y_pred))

In [ ]:
confusion_matrix(y_true, y_pred, sample_weight=None)

result on test set

In [ ]:
#restore test results
ts_pred_lin = np.zeros([4,len(test_X)]) 

In [ ]:
for i in range(4):
    theta = theta_lin[i]
    ts_pred_lin[i] = (test_X@theta).reshape(len(test_X))

In [ ]:
test_pred_lin = onevsall(ts_pred_lin)

In [ ]:
sum(test_pred_lin == test_data['Stance'])/ len(test_data)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = test_data['Stance']
y_pred = test_pred_lin
print(classification_report(y_true, y_pred))

In [ ]:
test_pred_lin = onevsall_new(ts_pred_lin,t)

In [ ]:
sum(test_pred_lin == test_data['Stance'])/ len(test_data)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = test_data['Stance']
y_pred = test_pred_lin
print(classification_report(y_true, y_pred))

In [ ]:
confusion_matrix(y_true, y_pred, sample_weight=None)

In [ ]:
col[6]

LR models

In [ ]:
theta_lr = []
cost_lr = []
lam = 0.1
model_names_lr = ['lr_unrelated','lr_agree','lr_disagree','lr_discuss']
#restore prediction on validation
y_pred_lr = np.zeros([4,len(valid_X)]) 

In [ ]:
#build 4 models
#unrelated
i = 0
model_name = model_names_lr[i]
print(model_name)
X = train_X
y = train[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_lr(X, y, ini_theta, learning_rate, iters, lam)
theta_lr.append(theta)
cost_lr.append(cost)
y_pred_lr[i] = (md.sigmoid(valid_X@theta)).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0.7
y_pred_class = y_pred_lr[i] > threshold
valid_y_class = valid_y > 0.5
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lr[i])

In [ ]:
#agree
i = 1
model_name = model_names_lr[i]
print(model_name)
#downsampling
# sample = downsampling(train, model_name, r = 1)
sample = downsampling(train, model_name, r = 0.3)
X = sample[xcols].as_matrix()
X = md.preprocess_X(X)
y = sample[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_lr(X, y, ini_theta, learning_rate, iters, lam)
theta_lr.append(theta)
cost_lr.append(cost)
y_pred_lr[i] = (md.sigmoid(valid_X@theta)).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0.3
y_pred_class = y_pred_lr[i] > threshold
valid_y_class = valid_y > 0.5
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lr[i])

In [ ]:
#disagree
i = 2
model_name = model_names_lr[i]
print(model_name)
#downsampling
# sample = downsampling(train, model_name, r = 1)
sample = downsampling(train, model_name, r = 0.1)
X = sample[xcols].as_matrix()
X = md.preprocess_X(X)
y = sample[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_lr(X, y, ini_theta, learning_rate, iters, lam)
theta_lr.append(theta)
cost_lr.append(cost)
y_pred_lr[i] = (md.sigmoid(valid_X@theta)).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0.2
y_pred_class = y_pred_lr[i] > threshold
valid_y_class = valid_y > 0.5
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lr[i])

In [ ]:
#discuss
i = 3
model_name = model_names_lr[i]
print(model_name)
X = train_X
y = train[model_name].values.reshape([len(X),1])
#training...
theta, cost = md.gradient_lr(X, y, ini_theta, learning_rate, iters, lam)
theta_lr.append(theta)
cost_lr.append(cost)
y_pred_lr[i] = (md.sigmoid(valid_X@theta)).reshape(len(valid_X))
valid_y = valid[model_name].values.reshape([len(valid),1])

In [ ]:
threshold = 0.2
y_pred_class = y_pred_lr[i] > threshold
valid_y_class = valid_y > 0.5
md.evaluate(valid_y_class, y_pred_class)

In [ ]:
md.roc(valid_y, y_pred_lr[i])

In [ ]:
plt.figure(figsize = (15, 5))
plt.scatter(range(len(theta_lin[0])-1),np.abs(theta_lr[0][1:]), color = 'blue', label = 'unrelated')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lr[1][1:]), color = 'pink', label = 'agree')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lr[2][1:]), color = 'yellow', label = 'disagree')
plt.scatter(range(len(theta_lin[1])-1),np.abs(theta_lr[3][1:]), color = 'orange', label = 'discuss')
plt.xlabel('Features')
plt.ylabel('Theta Values of LR Models')
plt.legend()
plt.show()

In [ ]:
t = [0.7,0.3,0.2,0.2]

In [ ]:
valid_pred_lr = onevsall(y_pred_lr)

In [ ]:
sum(valid_pred_lr == valid['Stance'])/ 4997

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = valid['Stance']
y_pred = valid_pred_lr
print(classification_report(y_true, y_pred))

In [ ]:
valid_pred_lr = onevsall_new(y_pred_lr,t)

In [ ]:
sum(valid_pred_lr == valid['Stance'])/ 4997

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = valid['Stance']
y_pred = valid_pred_lr
print(classification_report(y_true, y_pred))

In [ ]:
confusion_matrix(y_true, y_pred, sample_weight=None)

result on test set

In [ ]:
#restore test results
ts_pred_lr = np.zeros([4,len(test_X)]) 

In [ ]:
for i in range(4):
    theta = theta_lr[i]
    ts_pred_lr[i] = (md.sigmoid(test_X@theta)).reshape(len(test_X))

In [ ]:
test_pred_lr = onevsall(ts_pred_lr)

In [ ]:
sum(test_pred_lr == test_data['Stance'])/ len(test_data)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = test_data['Stance']
y_pred = test_pred_lr
print(classification_report(y_true, y_pred))

In [ ]:
test_pred_lr = onevsall_new(ts_pred_lr,t)

In [ ]:
sum(test_pred_lr == test_data['Stance'])/ len(test_data)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = test_data['Stance']
y_pred = test_pred_lr
print(classification_report(y_true, y_pred))

In [ ]:
confusion_matrix(y_true, y_pred, sample_weight=None)

Xgboost model

In [ ]:
import xgboost as xgb  

In [ ]:
sample0 = (train[train['Stance'] == 'unrelated'].copy()).sample(frac = 0.2)
sample1 = train[train['Stance'] != 'unrelated'].copy()

In [ ]:
sample = pd.concat([sample0, sample1],axis=0)

In [ ]:
sample.shape

In [ ]:
col = list(sample.columns)[4:16]

In [ ]:
col

In [ ]:
tr_X = sample[col].values.copy()
vl_X = valid[col].values.copy()
ts_X = test_data[col].values.copy()

In [ ]:
tr_Y = sample['Stance'].values.copy()
vl_Y = valid['Stance'].values.copy()
ts_Y = test_data['Stance'].values.copy()

In [ ]:
for i in range(len(tr_Y)):
    if tr_Y[i] == 'unrelated':
        tr_Y[i] = 0
    elif tr_Y[i] == 'agree':
        tr_Y[i] = 1
    elif tr_Y[i] == 'disagree':
        tr_Y[i] = 2
    elif tr_Y[i] == 'discuss':
        tr_Y[i] = 3

In [ ]:
for i in range(len(vl_Y)):
    if vl_Y[i] == 'unrelated':
        vl_Y[i] = 0
    elif vl_Y[i] == 'agree':
        vl_Y[i] = 1
    elif vl_Y[i] == 'disagree':
        vl_Y[i] = 2
    elif vl_Y[i] == 'discuss':
        vl_Y[i] = 3

In [ ]:
params={
'booster':'gbtree',
'objective': 'multi:softmax', 
'num_class':4, 
'gamma':0.05, 
'max_depth':15, 
# 'lambda':50, 
'subsample':0.3, 
'colsample_bytree':0.7, 
'silent':1 ,
'eta': 0.2, 
'seed':710,
'nthread':4,
}

plst = list(params.items())

num_rounds = 100 
xg_train = xgb.DMatrix(tr_X, label=tr_Y)
xg_val = xgb.DMatrix(vl_X, label=vl_Y)
xg_test = xgb.DMatrix(ts_X)
watchlist = [(xg_train, 'train'),(xg_val, 'val')]

# training...
model = xgb.train(plst, xg_train, num_rounds, watchlist,early_stopping_rounds=100)
preds = model.predict(xg_test,ntree_limit=model.best_iteration)

In [ ]:
set(preds)

In [ ]:
pred_class = []
for i in range(len(preds)):
    if preds[i] == 0:
        pred_class.append('unrelated')
    elif preds[i] == 1:
        pred_class.append('agree')
    elif preds[i] == 2:
        pred_class.append('disagree')
    elif preds[i] == 3:
        pred_class.append('discuss')
assert len(pred_class) == len(preds)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = ts_Y
y_pred = pred_class
print(classification_report(y_true, y_pred))

In [ ]:
confusion_matrix(y_true, y_pred, sample_weight=None)

In [ ]:
sum(y_pred == y_true)/len(y_true)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 8))
xgb.plot_importance(model, ax=ax, max_num_features=20)
plt.show()

In [ ]:
col[8]

In [ ]:
col[3]

In [ ]:
col[9]

In [ ]:
col[5]

In [ ]:
col[2]

In [ ]:
col[4]

In [ ]:
col[7]

In [ ]:
col[11]